#LP Problems

You should submit your python code as a solution. Use the PuLP library https://pypi.org/project/PuLP/ to solve the following problems. Documentation to PuLP can be found here: https://coin-or.github.io/pulp/main/index.html

A few hints
To surpress the messages from the solver, you can use `GLPK(msg = 0)` (if you use GLPK) or `PULP_CBC_CMD(msg=False)` (if you use CBC) or a similar option for other solvers as a solver parameter of `solve()` method. You can use function `value()` to evaluate any expression involving the variables in the optimal solution. If you need many variables, it might be good to introduce them using an array of their names using `dicts()` method and use the returned dictionary to access them.

###Part 1 (10 points)
Solve the following program:
```
       min 122x + 143y
subject to:
         x ≥ -10
         y ≤  10
  3x +  2y ≤  10
 12x + 14y ≥ -12.5
  2x +  3y ≥   3
  5x -  6y ≥-100

```
Required output example:
```
Optimal solution: x = 0.1 y = -2.3
Objective value: 100.1214
Tight constraints:
1
2
4
Unique optimal solution: NO
```
So, your program should find the optimal solution, determine its objective value, identify the tight constraints and decide whether the optimal solution is unique.
Note that, in $\mathbb{R}^2$, there are only two directions perpendicular to the objective.

###Part 2 (10 points)
Find an optimal mixed strategy of the following game: Both players choose independently a single integer from 1 to 6. Then, the numbers are compared:

If they are equal, there is a draw
If they differ by 1, the player who played the smaller number gets 2EUR from the other player
If they differ by ≥2, the player who played the larger number gets 1EUR from the other player
Note that the game is symmetric and the same strategy is optimal for both players.
Required output example:
```
x1: 0.2 x2: 0.1 x3: 0.2 x4: 0.1 x5: 0.2 x6: 0.2
```
###Part 3 (10 points)
On some imaginary island, there are 69 companies and there are bilateral contracts between them. The monarch of the island would like to inspect validity of each of these contracts during a single large event. The monarch requires two representatives to represent each contract relationship (they can be both from the same party of the contract or each from a different one). This is of course satisfied by each company sending a single representative, which would require involvement of 69 representatives in total. However, the companies want to find a solution which minimizes the total number of representatives who need to attend the event.

Input file hw1-03.txt contains information about the contracts. Each line corresponds to a single contract and contains identifiers (1-69) of both involved parties separated by a space.

Example output:
```
representatives from company 1: 1.0
representatives from company 2: 2.0
representatives from company 3: 1.0
...
representatives from company 69: 1.0
Total number of representatives involved: 58
```
**Important**: It is not possible to send an arbitrary fraction of a representative. However, it is enough to solve the LP relaxation since it already gives an integral solution. Reasons for this will be explained later in the class.

---

## Part 1

In [3]:
# Import PuLP
from pulp import *

# Create the problem
prob = LpProblem("Problem", LpMinimize)

#Create the variables
x = LpVariable("x", lowBound = -10, cat = "Continuous")
y = LpVariable("y", upBound = 10, cat = "Continuous")

# Objective function
prob += 122*x + 143*y

# Constraints
constraints=[0]*6
constraints[0] = x >= -10
constraints[1] = y <= 10
constraints[2] = 3*x + 2*y <= 10
constraints[3] = 12*x + 14*y >= -12.5
constraints[4] = 2*x + 3*y >= 3
constraints[5] = 5*x - 6*y >= -100

for i in range(6):
    prob += constraints[i]

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=False))

# Print the optimal solution
print(f"Optimal solution: x = {x.value()}, y = {y.value()}")

# Print its objective value
print(f"Objective value: {value(prob.objective)}")

# Print the tight constraints
print("Tight constraints:")
if value(x) == -10:
    print(1)
if value(y) == 10:
    print(2)
if 3*value(x) + 2*value(y) == 10:
    print(3)
if 12*value(x) + 14*value(y) == -12.5:
    print(4)
if 2*value(x) + 3*value(y) == 3:
    print(5)
if 5*value(x) - 6*value(y) == -100:
    print(6)


Optimal solution: x = -9.9375, y = 7.625
Objective value: -122.0
Tight constraints:
4
5


## Part 2

In [4]:
# Import PuLP and numpy
import numpy as np
from pulp import *

# Create matrix
rows, cols = 6,6
M = np.zeros((rows, cols))
for i in range (rows):
    for j in range (cols):
        if i==j:
            M[i,j] = 0
        elif i == j+1:
            M[i,j] = -2
        elif i > j+1:
            M[i,j] = 1
        elif i == j-1:
            M[i,j] = 2
        else:
            M[i,j] = -1

N = M.transpose()

# Create the problem
prob = LpProblem("Problem", LpMaximize)

# Create the variables
x0 = LpVariable("x0")
variables = np.array([LpVariable(f"x{i}", lowBound=0) for i in range(1,7)])

# Objective function
prob += x0

# Constraints
for i in range(6):
    prob += x0 <= variables*N[i]
prob += sum(variables) == 1

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=False))

# Print the optimal solutions
for i in range(6):
    print(f"x{i+1}: {variables[i].value()} ", end = '')

x1: 0.0 x2: 0.0625 x3: 0.3125 x4: 0.25 x5: 0.3125 x6: 0.0625 

##Part 3

In [6]:
# Import PuLP
from pulp import *
import numpy as np

# Create matrix
rows, cols = 69,69
M = np.zeros((rows,cols))
with open('hw1-03.txt') as f:
    for line in f:
        x, y = map(int, line.strip().split())
        M[x-1][y-1] = 1
        M[y-1][x-1] = 1

# Define the problem
prob = LpProblem("Minimum representatives problem", LpMinimize)

# Define the variables
representatives = np.array([LpVariable
                            (f"representative for company number {i+1}",
                             lowBound=0, cat = int) for i in range(69)])

# Define the objective function
prob += lpSum(representatives)

# Define the constraints

for i in range(69):
    prob += representatives[i] <= lpSum(M[i,j] for j in range(69))
for i in range(69):
    for j in range(69):
        if M[i,j] == 1:
            prob += representatives[i] + representatives[j] >= 2

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=False))

# Print the requested output
for i in range(69):
    print(f"representatives from company {i+1}: {representatives[i].value()}")

representatives from company 1: 1.0
representatives from company 2: 1.0
representatives from company 3: 1.0
representatives from company 4: 2.0
representatives from company 5: 1.0
representatives from company 6: 1.0
representatives from company 7: 1.0
representatives from company 8: 1.0
representatives from company 9: 1.0
representatives from company 10: 1.0
representatives from company 11: 1.0
representatives from company 12: 1.0
representatives from company 13: 1.0
representatives from company 14: 0.0
representatives from company 15: 1.0
representatives from company 16: 1.0
representatives from company 17: 1.0
representatives from company 18: 1.0
representatives from company 19: 1.0
representatives from company 20: 1.0
representatives from company 21: 1.0
representatives from company 22: 1.0
representatives from company 23: 1.0
representatives from company 24: 0.0
representatives from company 25: 1.0
representatives from company 26: 1.0
representatives from company 27: 1.0
representa

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
